In [5]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [19]:
import rpy2
#import rpy2.robjects as robjects
#import rpy2.robjects.packages as rpackages

#from rpy2.robjects import numpy2ri, pandas2ri


import numpy as np
import pandas as pd
from rpy2.robjects import r, rl, pandas2ri, numpy2ri
from rpy2.robjects.packages import importr


print(rpy2.__version__)

r('''
library(MetENP)
''')

3.4.2


'MetENP','tools','stats',...,'datasets','methods','base'


In [20]:
numpy2ri.activate()
pandas2ri.activate()

#numpy2ri.deactivate()
#pandas2ri.deactivate()


#bring this variable into R
#r['study_id'] = {{ study_id }}
#r('head(study_id)')

In [21]:
%%appyter hide_code
{% do SectionField(
    name='INPUT',
    title='Input Datasets',
    subtitle='Add your datasets here',
)%}
{% do SectionField(
    name='SIGM',
    title='Find Significant Metabolites',
    subtitle='Run significance of all the analysis summary together',
)%}
{% do SectionField(
    name='COUNTCLASS',
    title='Count Metabolites by Class',
    subtitle='Count the metabolites of each class',
)%}
{% do SectionField(
    name='ENRICH',
    title='Enrichment Score by Class',
    subtitle='Calculate the enrichment score of each metabolite class',
)%}
{% do SectionField(
    name='HEAT',
    title='Heatmap',
    subtitle='Heatmap of metabolites x enriched pathways',
)%}
{% do SectionField(
    name='DOT',
    title='Dotplot',
    subtitle='Plot a dotplot of enrichment by class',
)%}


In [22]:
%%appyter hide_code

{% set study_id = StringField(name='study_id', label='DatasetID', description = "Type your datasetID here", default='ST000915', section='INPUT') %}

In [23]:
%%appyter hide_code 

{% set studyOrg = StringField(name='studyOrg', label='KEGG Organism Code', description = "hsa for human, mmu for mouse, eco for E. coli. See https://www.genome.jp/kegg/catalog/org_list.html for all organisms", default='hsa', section='INPUT') %}

In [24]:
%%appyter hide_code

{% set countclass = MultiChoiceField(
    name='countclass',
    label='Select Class Type',
    description='Choose between sub_class, main_class and super_class',
    default=[
        'sub_class',
    ],
    choices=[
        'sub_class',
        'main_class',
        'super_class',
    ],
    section='INPUT'
) %}

In [25]:
%%appyter code_exec

r.assign('study_id', {{study_id}})


```python

r.assign('study_id', 'ST000915')
```

In [26]:
r('''
typeof(study_id)
study_id
''')
#study_id

'ST000915'


In [36]:
r('''
print(typeof(study_id))
print("here")
data_1 = getmwstudies(study_id, 'data')
''')

[1] "character"
[1] "here"


ValueError: array-shape mismatch in array 5

In [37]:
%%appyter code_exec

data_1 = r['getmwstudies']({{
     study_id}}, 
    'data'
)

```python
data_1 = r['getmwstudies']('ST000915',
    'data'
)
```

In [4]:
#TODO: outputs are not the same type as the original vignette
#r['head'](data_1)

r.assign('data_1', data_1)
r('''
head(data_1)
''')


NameError: name 'data_1' is not defined

In [10]:
#robjects.r['is.data.frame'](data_1)
#print(type(data_1))

In [38]:
%%appyter code_exec

metadata = r['getmwstudies']({{
     study_id}}, 
    'factors'
)

```python
metadata = r['getmwstudies']('ST000915',
    'factors'
)
```

In [86]:
%%appyter code_exec

metadata = r['getmwstudies']({{
     study_id}}, 
    'factors'
)


```python
metadata = r['getmwstudies']('ST000915',
    'factors'
)
```

In [12]:
robjects.r['head'](metadata)

study_id,local_sample_id,subject_type,factors,Diagnosis
...,...,...,...,...


Find the factors you would want to compare
Multiple factors (independent variables) are formatted in multiple columns but you can get information on all the factors by "factors" column, such as 'TreatmentGroup:Prednisolone', this indicates column name 'TreatmentGroup' with factor Prednisolone. For t-test use the factors in the same column. For comparing multipe independent variables use anova by anova_ana function.

### Associate metabolomics data to the refmet class

In [13]:
refmet_class = robjects.r['convert_refmet'](data_1)


In [14]:
robjects.r['head'](refmet_class)

#saving df as an obj in r2py

metabolite_name,analysis_id,analysis_summary,...,super_class,main_class,sub_class
"'1,2-...",StrSe...,StrSe...,...,'Glyc...,'Dira...,'DAG'
"'1,2-...",StrSe...,StrSe...,,'Glyc...,'Dira...,'DAG'
"'1,2-...",StrSe...,StrSe...,,'Glyc...,'Dira...,'DAG'
"'1,2-...",StrSe...,StrSe...,,'Glyc...,'Dira...,'DAG'
"'1,2-...",StrSe...,StrSe...,,'Glyc...,'Dira...,'DAG'
"'1,2-...",StrSe...,StrSe...,,'Glyc...,'Dira...,'DAG'


In [15]:
metadata_f = metadata.rx2("factors")
unique_metadata_f = robjects.r["unique"](metadata_f)
#should be able to select any number of these for analysis with significant_met

In [16]:
print(unique_metadata_f)

[[1]]
[1] "Diagnosis:Cirrhosis"

[[2]]
[1] "Diagnosis:NASH"

[[3]]
[1] "Diagnosis:Normal"

[[4]]
[1] "Diagnosis:Steatosis"




In [17]:
data_1_as = data_1.rx2("analysis_summary")
unique_data_1_as = robjects.r["unique"](data_1_as)

In [18]:
print(unique_data_1_as)

[[1]]
[1] "Core G Fatty acids/Eicosanoids"

[[2]]
[1] "Core J Sterols"

[[3]]
[1] "Core K Prenols/Cardiolipins"

[[4]]
[1] "Core E Neutral Lipids"

[[5]]
[1] "Core I Sphingolipids"

[[6]]
[1] "Core H Phospholipids"




### Find significant metabolites, run significance of all the analysis summary together. The analysis summary/modes you got in the previous section

There is a normalization step which takes into account missing values, and this can be achieved via three methods:

1) half_of_min: where the NAs are replaced by half of min values in the data

2) remove_NAs: where Cols with NAs values are removed

3) 50percent: where cols with more than 50% NAs values are removed

half_of_min is ideal when you wish to see which metabolites were present in either group. Very high fold change would mean it was present in either group.

In [19]:
%%appyter code_eval

{% set normtype = MultiChoiceField(
    name='normtype',
    label='Select Type of Normalization',
    description='Select 1. half_of_min: where the NAs are replaced by half of min values in the data, 2. remove_NAs: where Cols with NAs values are removed, or 3. 50percent: where cols with more than 50% NAs values are removed',
    default=[
        '50percent',
    ],
    choices=[
        'half_of_min',
        'remove_NAs',
        '50percent',
    ],
    section='SIGM'
) %}


In [20]:
%%appyter code_eval

{% set pthres = StringField(
    name = 'pthres',
    label = 'P-Value Threshhold',
    description= "For Volcano Plot. Float between 0 and 1.",
    default='0.05',
    section='SIGM'
) %}

In [21]:
%%appyter code_eval
#need to change to float
robjects.r['print']({{pthres}})

```python
#need to change to float
robjects.r['print']('0.05')
```

[1] "0.05"


'0.05'


In [24]:
%%appyter code_eval

thres_log2foldchange = {{ StringField(
    name = 'tres_log2foldchange',
    label = 'Log2FoldChange Threshhold',
    description= "For Volcano Plot. Float between 0 and 1.",
    default='0.5',
    section='SIGM'
) }}
thres_log2foldchange = float(thres_log2foldchange)
thres_log2foldchange

```python
thres_log2foldchange = '0.5'
thres_log2foldchange = float(thres_log2foldchange)
thres_log2foldchange
```

0.5

In [ ]:
%%appyter code_eval

{% set pthres = StringField(
    name = 'pthres',
    label = 'P-Value Threshhold',
    description= "For Volcano Plot. Float between 0 and 1.",
    default='0.05',
    section='SIGM'
) %}

In [27]:
%%appyter code_eval



robjects.r["significant_met"](refmet_class, "metabolite_name", ("Core G Fatty acids/Eicosanoids","Core J Sterols","Core K Prenols/Cardiolipins","Core E Neutral Lipids","Core I Sphingolipids","Core H Phospholipids"), metadata, 'Cirrhosis', 'Normal', 'Diagnosis', 'local_sample_id', 'fdr',{{normtype}})


```python

robjects.r["significant_met"](refmet_class, "metabolite_name", ("Core G Fatty acids/Eicosanoids","Core J Sterols","Core K Prenols/Cardiolipins","Core E Neutral Lipids","Core I Sphingolipids","Core H Phospholipids"), metadata, 'Cirrhosis', 'Normal', 'Diagnosis', 'local_sample_id', 'fdr',['50percent'])
```

In [105]:
%%appyter code_eval

thres_log2foldchange_countclass = {{ StringField(
    name = 'tres_log2foldchange_countclass',
    label = 'Log2FoldChange Threshhold',
    description= "Float between 0 and 1.",
    default='0.5',
    section='COUNTCLASS'
) }}
thres_log2foldchange_countclass = float(thres_log2foldchange_countclass)
thres_log2foldchange_countclass

```python
thres_log2foldchange_countclass = '0.5'
thres_log2foldchange_countclass = float(thres_log2foldchange_countclass)
thres_log2foldchange_countclass
```

0.5

In [106]:
%%appyter code_eval

sigmets = {{ IntField(
    name = 'sigmets',
    label = 'Number of Significant Metabolites per Class',
    description= "Number of significant metabolies in a class; suggested to be 3 or more.",
    default=3,
    section = 'ENRICH'
) }}
sigmets

```python
sigmets = 3
sigmets
```

3

In [107]:
%%appyter code_eval

x_size_heat = {{ IntField(
    name = 'x_size',
    label = 'Font Size for X',
    description= "To change the size of the heatmap",
    default=8,
    section = 'HEAT'
) }}
x_size_heat

```python
x_size_heat = 8
x_size_heat
```

8

In [108]:
%%appyter code_eval

y_size_heat = {{ IntField(
    name = 'y_size_heat',
    label = 'Font Size for Y',
    description= "To change the size of the heatmap",
    default=6,
    section = 'HEAT'
) }}
y_size_heat

```python
y_size_heat = 6
y_size_heat
```

6

In [109]:
%%appyter code_eval

x_size_dot = {{ IntField(
    name = 'x_size_dot',
    label = 'Font Size for X',
    description= "To change the size of the dotplot",
    default=8,
    section = 'DOT'
) }}
x_size_dot

```python
x_size = 8
x_size
```

8

In [110]:
%%appyter code_eval

y_size_dot = {{ IntField(
    name = 'y_size_dot',
    label = 'Font Size for Y',
    description= "To change the size of the dotplot",
    default=6,
    section = 'DOT'
) }}
y_size_dot

```python
y_size = 6
y_size
```

6